## Exercise: Fill out Preprocess Step


In [7]:
%%bash
pip install stepfunctions
# pip install stepfunctions==2.2.0

  Using cached stepfunctions-2.3.0.tar.gz (67 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
nfo did not run successfully. egg_i
  │ exit code: 1
 lines of output]86
ges/setuptools/dist.py:599: SetuptoolsDeprecationWarning: Invalid dash-separated key 'description-file' in 'metadata' (setup.cfg), please use the underscore name 'description_file' instead.
 !!   
      
****************************************************************************
ot be supported in futureage of dash-separated 'description-file' will n
on_file' instead.      versions. Please use the underscore name 'descripti
      
ject and remove deprecated callsar-03, you need to update your pro
              or your builds will no longer be supported.
      
e_config.html for details. https://setuptools.pypa.io/en/latest/userguide/declarativ
*************************************************************************
      
      !!
e_underscore(opt, section)f._enforc
packages/setuptools/dist.py:599: SetuptoolsDeprecationWarning: Invalid dash-separated key 'description-file

CalledProcessError: Command 'b'pip install stepfunctions\n# pip install stepfunctions==2.2.0\n'' returned non-zero exit status 1.

In [59]:
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from stepfunctions.steps.sagemaker import ProcessingStep
import sagemaker

role = get_execution_role()

PREPROCESSING_JOB_NAME = 'test-job6-preprocess'
input_data = 's3://udacity-sagemaker-solutiondata2021/l3e1/reviews_Musical_Instruments_5.json.zip'
input_preprocessing_code = 's3://udacity-sagemaker-solutiondata2021/l3e3/HelloBlazePreprocess.py'
sess = sagemaker.Session()

sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.m5.large',
                                     instance_count=1)


processed_data_train = "{}{}/{}".format("s3://", sess.default_bucket(), 'hello_blaze_train_scikit')
processed_data_test = "{}{}/{}".format("s3://", sess.default_bucket(), 'hello_blaze_test_scikit')

inputs=[ProcessingInput(source=input_data, destination='/opt/ml/processing/input', input_name = 'input-1'),  ProcessingInput(source=input_preprocessing_code , destination='/opt/ml/processing/input/code', input_name = 'code')]


outputs=[ProcessingOutput(source='/opt/ml/processing/output/train', destination=processed_data_train, output_name = 'train_data'), ProcessingOutput(source='/opt/ml/processing/output/test', destination=processed_data_test, output_name = 'test_data')]


processing_step = ProcessingStep(
    "SageMaker pre-processing step 4",
    processor=sklearn_processor,
    job_name=PREPROCESSING_JOB_NAME,
    inputs=inputs,
    outputs=outputs,
    container_entrypoint=["python3", "/opt/ml/processing/input/code/HelloBlazePreprocess.py"],
)



print(" ".join([processed_data_train, processed_data_test]))
print(inputs)


s3://sagemaker-us-west-2-565094796913/hello_blaze_train_scikit s3://sagemaker-us-west-2-565094796913/hello_blaze_test_scikit
[<sagemaker.processing.ProcessingInput object at 0x1203172e0>, <sagemaker.processing.ProcessingInput object at 0x121221850>]


## Exercise: Fill out Training Step

In [60]:
from stepfunctions.steps.sagemaker import TrainingStep
import boto3

WORKFLOW_OUTPUT = "s3://udacity-sagemaker-solutiondata2021/l3e3/workflow_output"
TRAINING_JOB_NAME = "test-job2-train"

region_name = boto3.Session().region_name
container = sagemaker.image_uris.retrieve(
    region=region_name, framework="blazingtext", version="latest"
)

helloBlazeEstimator = sagemaker.estimator.Estimator(
    container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    volume_size=30,
    max_run=360000,
    input_mode="File",
    output_path=WORKFLOW_OUTPUT,
    sagemaker_session=sess,
)

helloBlazeEstimator.set_hyperparameters(mode='supervised')

training_step = TrainingStep(
    "SageMaker Training Step",
    estimator=helloBlazeEstimator,
    data={"train": sagemaker.TrainingInput(processed_data_train, content_type="text/plain"), "validation": sagemaker.TrainingInput(processed_data_test, content_type="text/plain")},
    job_name=TRAINING_JOB_NAME,
    wait_for_completion=True,
)

## Exercise: Create & Execute Workflow

In [63]:
from stepfunctions.steps import Chain
from stepfunctions.workflow import Workflow

workflow_role = 'arn:aws:iam::565094796913:role/step-function-execution-role'

workflow_graph = Chain([processing_step, training_step])
workflow = Workflow(
    name="SageMakerProcessingWorkflow7",
    definition=workflow_graph,
    role=workflow_role,
)

workflow.create()

# Execute workflow
execution = workflow.execute(
    inputs={
        "PreprocessingJobName": PREPROCESSING_JOB_NAME,  # Each pre processing job (SageMaker processing job) requires a unique name,
        "TrainingJobName": TRAINING_JOB_NAME  # Each Sagemaker Training job requires a unique name,       
    }
)

execution_output = execution.get_output(wait=True)


In [64]:
execution.render_progress()